In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [71]:
movies = pd.read_csv(r'D:\Project\Movie Recommendation\data\raw\tmdb_5000_movies.csv')
credits = pd.read_csv(r'D:\Project\Movie Recommendation\data\raw\tmdb_5000_credits.csv')

In [72]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [73]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [74]:
df = movies.merge(credits,left_on='id',right_on='movie_id')

In [75]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [77]:
df['production_countries'].value_counts()

production_countries
[{"iso_3166_1": "US", "name": "United States of America"}]                                                                                                                                     2977
[{"iso_3166_1": "GB", "name": "United Kingdom"}, {"iso_3166_1": "US", "name": "United States of America"}]                                                                                      181
[]                                                                                                                                                                                              174
[{"iso_3166_1": "GB", "name": "United Kingdom"}]                                                                                                                                                131
[{"iso_3166_1": "DE", "name": "Germany"}, {"iso_3166_1": "US", "name": "United States of America"}]                                                                                             119

In [78]:
df['keywords'].values[0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [79]:
df['spoken_languages'].values[0]

'[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "es", "name": "Espa\\u00f1ol"}]'

In [80]:
df['spoken_languages'].values[0]

'[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "es", "name": "Espa\\u00f1ol"}]'

In [81]:
content_df = df[['movie_id','title_x','overview','genres','keywords','tagline','spoken_languages','production_companies','cast','crew']]

In [82]:
content_df.head(2)

,movie_id,title_x,overview,genres,keywords,tagline,spoken_languages,production_companies,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Enter the World of Pandora.,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","At the end of the world, the adventure begins.","[{""iso_639_1"": ""en"", ""name"": ""English""}]","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [83]:
content_df.isnull().sum()

movie_id                  0
title_x                   0
overview                  3
genres                    0
keywords                  0
tagline                 844
spoken_languages          0
production_companies      0
cast                      0
crew                      0
dtype: int64

In [84]:
content_df['tagline'].fillna('',inplace=True)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\1258946343.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  content_df['tagline'].fillna('',inplace=True)
C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\1258946343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['tagline'].fillna('',inplace=True)


In [85]:
content_df.isnull().sum()

movie_id                0
title_x                 0
overview                3
genres                  0
keywords                0
tagline                 0
spoken_languages        0
production_companies    0
cast                    0
crew                    0
dtype: int64

In [86]:
content_df.dropna(inplace=True)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\1671458372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df.dropna(inplace=True)


In [87]:
content_df.isnull().sum()

movie_id                0
title_x                 0
overview                0
genres                  0
keywords                0
tagline                 0
spoken_languages        0
production_companies    0
cast                    0
crew                    0
dtype: int64

In [88]:
content_df.duplicated().sum()  

0

In [89]:
content_df.head(2)

,movie_id,title_x,overview,genres,keywords,tagline,spoken_languages,production_companies,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Enter the World of Pandora.,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","At the end of the world, the adventure begins.","[{""iso_639_1"": ""en"", ""name"": ""English""}]","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [90]:
def select_top3(text):
    L = []
    K = ast.literal_eval(text)
    i = 0
    for i in range(len(K)):
        if i<3:
            L.append(K[i]['name'])
            i += 1
        else:
            break
    return L

In [91]:
content_df['genres'] = content_df['genres'].apply(select_top3)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\592084733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['genres'] = content_df['genres'].apply(select_top3)


In [92]:
content_df['keywords'] = content_df['keywords'].apply(select_top3)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\334674564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['keywords'] = content_df['keywords'].apply(select_top3)


In [93]:
content_df['spoken_languages'].values[0]

'[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "es", "name": "Espa\\u00f1ol"}]'

In [94]:
def select_one(text):
    L = []
    K = ast.literal_eval(text)
    i = 0
    for i in range(len(K)):
        if i<1:
            L.append(K[i]['name'])
            i += 1
        else:
            break
    return L

In [95]:
content_df['spoken_languages'] = content_df['spoken_languages'].apply(select_one)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\3844203124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['spoken_languages'] = content_df['spoken_languages'].apply(select_one)


In [96]:
content_df['spoken_languages'].value_counts()

spoken_languages
[English]             4101
[Français]             108
[]                      85
[Español]               83
[Deutsch]               61
[العربية]               33
[Italiano]              32
[普通话]                   32
[Pусский]               31
[Český]                 30
[广州话 / 廣州話]             28
[日本語]                   23
[हिन्दी]                22
[Português]             17
[]                      12
[Dansk]                 12
[한국어/조선말]                8
[Latin]                  8
[Nederlands]             6
[עִבְרִית]               6
[Afrikaans]              5
[ελληνικά]               5
[svenska]                5
[Norsk]                  4
[Română]                 4
[Magyar]                 4
[ภาษาไทย]                4
[Polski]                 3
[فارسی]                  3
[Kiswahili]              3
[Український]            2
[தமிழ்]                  2
[No Language]            2
[Tiếng Việt]             2
[Gaeilge]                1
[اردو]                   1
[Bahasa ind

In [97]:
content_df.drop('spoken_languages',axis=1,inplace=True)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\1840280344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df.drop('spoken_languages',axis=1,inplace=True)


In [98]:
content_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 0 to 4802
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   movie_id              4800 non-null   int64 
 1   title_x               4800 non-null   object
 2   overview              4800 non-null   object
 3   genres                4800 non-null   object
 4   keywords              4800 non-null   object
 5   tagline               4800 non-null   object
 6   production_companies  4800 non-null   object
 7   cast                  4800 non-null   object
 8   crew                  4800 non-null   object
dtypes: int64(1), object(8)
memory usage: 375.0+ KB


In [99]:
content_df['production_companies'].values[0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [100]:
content_df['production_companies'] = content_df['production_companies'].apply(select_one)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\3458705321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['production_companies'] = content_df['production_companies'].apply(select_one)


In [101]:
content_df['cast'].values[0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [102]:
content_df['cast'] = df['cast'].apply(select_top3)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\2005934241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['cast'] = df['cast'].apply(select_top3)


In [103]:
def select_crew(text):
    L = []
    K = ast.literal_eval(text)
    for i in K:
        if i['job'] == 'Director' or i['job'] == 'Screenplay' or i['job'] == 'Writer':
            L.append(i['name'])
    return L

In [104]:
content_df['crew'] = content_df['crew'].apply(select_crew)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\2930821986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['crew'] = content_df['crew'].apply(select_crew)


In [105]:
content_df.rename(columns={'title_x':'title'},inplace=True)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\1117446502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df.rename(columns={'title_x':'title'},inplace=True)


In [106]:
content_df.head(2)

,movie_id,title,overview,genres,keywords,tagline,production_companies,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy]","[culture clash, future, space war]",Enter the World of Pandora.,[Ingenious Film Partners],"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[James Cameron, James Cameron, James Cameron]"
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island]","At the end of the world, the adventure begins.",[Walt Disney Pictures],"[Johnny Depp, Orlando Bloom, Keira Knightley]","[Gore Verbinski, Ted Elliott, Terry Rossio]"


In [107]:
def remove_space(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [108]:
content_df['genres'] = content_df['genres'].apply(remove_space)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\3606349657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['genres'] = content_df['genres'].apply(remove_space)


In [109]:
content_df['keywords'] = content_df['keywords'].apply(remove_space)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\3280542348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['keywords'] = content_df['keywords'].apply(remove_space)


In [110]:
content_df['production_companies'] = content_df['production_companies'].apply(remove_space)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\4051636051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['production_companies'] = content_df['production_companies'].apply(remove_space)


In [111]:
content_df['cast'] = content_df['cast'].apply(remove_space)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\743229747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['cast'] = content_df['cast'].apply(remove_space)


In [112]:
content_df['crew'] = content_df['crew'].apply(remove_space)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\3659247561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['crew'] = content_df['crew'].apply(remove_space)


In [113]:
content_df.head(2)

,movie_id,title,overview,genres,keywords,tagline,production_companies,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy]","[cultureclash, future, spacewar]",Enter the World of Pandora.,[IngeniousFilmPartners],"[SamWorthington, ZoeSaldana, SigourneyWeaver]","[JamesCameron, JamesCameron, JamesCameron]"
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland]","At the end of the world, the adventure begins.",[WaltDisneyPictures],"[JohnnyDepp, OrlandoBloom, KeiraKnightley]","[GoreVerbinski, TedElliott, TerryRossio]"


In [114]:
def make_each_word_list(text):
    return [text]

In [115]:
content_df['overview'] = content_df['overview'].apply(make_each_word_list)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\906149045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['overview'] = content_df['overview'].apply(make_each_word_list)


In [116]:
content_df['tagline'] = content_df['tagline'].apply(make_each_word_list)

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\3431220630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['tagline'] = content_df['tagline'].apply(make_each_word_list)


In [117]:
content_df.head(2)

,movie_id,title,overview,genres,keywords,tagline,production_companies,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy]","[cultureclash, future, spacewar]",[Enter the World of Pandora.],[IngeniousFilmPartners],"[SamWorthington, ZoeSaldana, SigourneyWeaver]","[JamesCameron, JamesCameron, JamesCameron]"
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland]","[At the end of the world, the adventure begins.]",[WaltDisneyPictures],"[JohnnyDepp, OrlandoBloom, KeiraKnightley]","[GoreVerbinski, TedElliott, TerryRossio]"


In [118]:
content_df['tags'] = content_df['overview'] +content_df['genres'] + content_df['keywords'] + content_df['tagline'] + content_df['production_companies'] + content_df['cast'] + content_df['crew']

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\1006341193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['tags'] = content_df['overview'] +content_df['genres'] + content_df['keywords'] + content_df['tagline'] + content_df['production_companies'] + content_df['cast'] + content_df['crew']


In [119]:
content_df['tags'].values[0]

['In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'cultureclash',
 'future',
 'spacewar',
 'Enter the World of Pandora.',
 'IngeniousFilmPartners',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'JamesCameron',
 'JamesCameron']

In [120]:
content_df['tags'] = content_df['tags'].apply(lambda x: " ".join(x))

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\2324165919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['tags'] = content_df['tags'].apply(lambda x: " ".join(x))


In [121]:
content_df['tags'].values[0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy cultureclash future spacewar Enter the World of Pandora. IngeniousFilmPartners SamWorthington ZoeSaldana SigourneyWeaver JamesCameron JamesCameron JamesCameron'

In [122]:
stemmer = SnowballStemmer('english')
content_df['tags'] = content_df['tags'].apply(lambda x: " ".join([stemmer.stem(i) for i in x.split()]))

C:\Users\Srinivas\AppData\Local\Temp\ipykernel_14320\3387929617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['tags'] = content_df['tags'].apply(lambda x: " ".join([stemmer.stem(i) for i in x.split()]))


In [123]:
content_df['tags'].values[0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi cultureclash futur spacewar enter the world of pandora. ingeniousfilmpartn samworthington zoesaldana sigourneyweav jamescameron jamescameron jamescameron'

In [124]:
content_df.head(2)

,movie_id,title,overview,genres,keywords,tagline,production_companies,cast,crew,tags
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy]","[cultureclash, future, spacewar]",[Enter the World of Pandora.],[IngeniousFilmPartners],"[SamWorthington, ZoeSaldana, SigourneyWeaver]","[JamesCameron, JamesCameron, JamesCameron]","in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland]","[At the end of the world, the adventure begins.]",[WaltDisneyPictures],"[JohnnyDepp, OrlandoBloom, KeiraKnightley]","[GoreVerbinski, TedElliott, TerryRossio]","captain barbossa, long believ to be dead, has ..."


In [125]:
content_df = content_df[['movie_id','title','tags']]

In [126]:
content_df.head(2)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, has ..."


In [133]:
vectorizer = TfidfVectorizer(
    stop_words="english",   
    ngram_range=(1,2),      
    max_features=5000   
)
tfidf_matrix = vectorizer.fit_transform(content_df["tags"])
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

In [134]:
cos_sim = cosine_similarity(tfidf_matrix)

In [135]:
cosine_df = pd.DataFrame(cos_sim, index=content_df['title'], columns=content_df['title'])

In [136]:
cosine_df.head()

title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On The Downlow,Sanctuary: Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,Newlyweds,"Signed, Sealed, Delivered",Shanghai Calling,My Date with Drew
title,,,,,,,,,,,,,,,,,,,,,
Avatar,1.000000,0.051205,0.021786,0.034554,0.038806,0.044009,0.020390,0.048458,0.038351,0.051465,...,0.010849,0.0,0.036015,0.029889,0.000000,0.003761,0.0,0.022507,0.000000,0.000000
Pirates of the Caribbean: At World's End,0.051205,1.000000,0.018628,0.013191,0.071250,0.049870,0.029362,0.044673,0.051513,0.053962,...,0.012498,0.0,0.016701,0.000000,0.000000,0.013126,0.0,0.020188,0.012353,0.000000
Spectre,0.021786,0.018628,1.000000,0.014402,0.038457,0.051003,0.009254,0.056497,0.067134,0.035905,...,0.061340,0.0,0.000000,0.000000,0.015846,0.024623,0.0,0.015829,0.000000,0.000000
The Dark Knight Rises,0.034554,0.013191,0.014402,1.000000,0.009295,0.025917,0.032249,0.028906,0.037482,0.176543,...,0.003053,0.0,0.044340,0.001936,0.000000,0.024367,0.0,0.017370,0.061320,0.044695
John Carter,0.038806,0.071250,0.038457,0.009295,1.000000,0.020093,0.028967,0.093985,0.006723,0.043630,...,0.022921,0.0,0.000000,0.009842,0.000000,0.003973,0.0,0.029101,0.000000,0.000000


In [137]:
pickle.dump(cosine_df,open(r'data\processed\cosine.pkl','wb'))

In [138]:
pickle.dump(content_df,open(r'data\processed\content.pkl', 'wb'))